# NFL Radio Affiliates: Seahawks vs Patriots
### SBE Chapter 15 Demo — What Can You Do With Python + Broadcast Data?

In this notebook we'll explore two CSV files exported from the **Radioland** broadcast database:
- `seahawks_affiliates.csv` — all AM & FM stations carrying Seattle Seahawks games
- `patriots_affiliates.csv` — all AM & FM stations carrying New England Patriots games

We'll walk through **8 steps** — loading data, counting stations, comparing AM vs FM,
mapping affiliate locations, and more. No prior Python experience required!

> **How to run:** Click each code cell and press **Shift + Enter** (or the Play button) to execute it.

---
## Step 1 — Load the CSV Files From GitHub

Pandas can read a CSV straight from a URL — no downloads needed.
We point it at the raw GitHub links and we're off.

In [ ]:
import pandas as pd

SEAHAWKS_URL = 'https://raw.githubusercontent.com/W2NJL/sbe15-demo/main/seahawks_affiliates.csv'
PATRIOTS_URL = 'https://raw.githubusercontent.com/W2NJL/sbe15-demo/main/patriots_affiliates.csv'

seahawks_raw = pd.read_csv(SEAHAWKS_URL)
patriots_raw = pd.read_csv(PATRIOTS_URL)

print(f'Seahawks rows loaded: {len(seahawks_raw)}')
print(f'Patriots rows loaded: {len(patriots_raw)}')
print()
print('Columns:', list(seahawks_raw.columns))

---
## Step 2 — Clean the Data (Deduplication)

In my RadioLand database, AM stations often have **multiple rows** — one for their daytime pattern and one
for nighttime (different power levels, sometimes different antenna arrays).
We'll keep one row per station, using the **highest power** value so our
counts and charts reflect unique stations.

This is a very common first step in data analysis — real-world data is
almost never perfectly clean out of the box.

In [ ]:
def dedup(df):
    """Keep one row per station (callsign + band), choosing the highest power."""
    df = df.copy()
    df['erp'] = pd.to_numeric(df['erp'], errors='coerce')
    df = df.sort_values('erp', ascending=False)
    df = df.drop_duplicates(subset=['callsign', 'band'], keep='first')
    return df.sort_values(['band', 'callsign']).reset_index(drop=True)

seahawks = dedup(seahawks_raw)
patriots = dedup(patriots_raw)

print(f'Seahawks unique stations: {len(seahawks)}  (was {len(seahawks_raw)} rows)')
print(f'Patriots unique stations: {len(patriots)}  (was {len(patriots_raw)} rows)')
print()

# Preview the Seahawks data
seahawks[['callsign', 'frequency', 'city', 'state', 'erp', 'format', 'band']].head(10)

---
## Step 3 — Which Team Has More Affiliates?

A simple bar chart tells the story at a glance. **Matplotlib** is Python's
core charting library — virtually every chart you see in data science starts here.

In [ ]:
import matplotlib.pyplot as plt

teams = ['Seahawks', 'Patriots']
counts = [len(seahawks), len(patriots)]

fig, ax = plt.subplots(figsize=(6, 4))
bars = ax.bar(teams, counts, color=['#69BE28', '#C60C30'], edgecolor='black', width=0.5)
ax.set_ylabel('Number of Stations')
ax.set_title('Total Radio Affiliates: Seahawks vs Patriots')

for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            str(count), ha='center', va='bottom', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

---
## Step 4 — AM vs FM Breakdown

How does each team split between the AM and FM bands?  This is a *grouped*
bar chart — we break each team's total into AM and FM side by side.

In [ ]:
import numpy as np

sea_am = len(seahawks[seahawks['band'] == 'AM'])
sea_fm = len(seahawks[seahawks['band'] == 'FM'])
pat_am = len(patriots[patriots['band'] == 'AM'])
pat_fm = len(patriots[patriots['band'] == 'FM'])

x = np.arange(2)
width = 0.3

fig, ax = plt.subplots(figsize=(7, 4))
am_bars = ax.bar(x - width/2, [sea_am, pat_am], width, label='AM', color='#FFD700', edgecolor='black')
fm_bars = ax.bar(x + width/2, [sea_fm, pat_fm], width, label='FM', color='#1E90FF', edgecolor='black')

ax.set_xticks(x)
ax.set_xticklabels(['Seahawks', 'Patriots'])
ax.set_ylabel('Number of Stations')
ax.set_title('AM vs FM Affiliates by Team')
ax.legend()

for bars in [am_bars, fm_bars]:
    for bar in bars:
        h = bar.get_height()
        if h > 0:
            ax.text(bar.get_x() + bar.get_width()/2, h + 0.3,
                    str(int(h)), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f'Seahawks — AM: {sea_am}, FM: {sea_fm}')
print(f'Patriots — AM: {pat_am}, FM: {pat_fm}')

---
## Step 5 — States With the Most Affiliates

Which states have the highest concentration of affiliates for each team?
`.value_counts()` is one of the most useful Pandas methods — it counts
how many times each unique value appears in a column.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sea_states = seahawks['state'].value_counts()
sea_states.plot(kind='barh', ax=axes[0], color='#69BE28', edgecolor='black')
axes[0].set_title('Seahawks Affiliates by State')
axes[0].set_xlabel('Number of Stations')
axes[0].invert_yaxis()

pat_states = patriots['state'].value_counts()
pat_states.plot(kind='barh', ax=axes[1], color='#C60C30', edgecolor='black')
axes[1].set_title('Patriots Affiliates by State')
axes[1].set_xlabel('Number of Stations')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print(f'Seahawks span {seahawks["state"].nunique()} states: {", ".join(sea_states.index)}')
print(f'Patriots span {patriots["state"].nunique()} states: {", ".join(pat_states.index)}')

---
## Step 6 — Interactive Map With Folium

**Folium** renders [Leaflet.js](https://leafletjs.com/) maps right inside the
notebook. Each station gets a color-coded marker — click one to see its
callsign, frequency, city, and power.

Folium comes **pre-installed** on Google Colab, so no `pip install` needed.

In [ ]:
import folium

# Center the map on the continental US
m = folium.Map(location=[44.5, -98.0], zoom_start=4, tiles='CartoDB positron')

def add_markers(df, color):
    for _, row in df.iterrows():
        if pd.isna(row['lat']) or pd.isna(row['lon']):
            continue
        # Longitude is stored as a positive number — negate it for the map
        lon = -abs(row['lon'])
        popup_html = (
            f"<b>{row['callsign']}</b><br>"
            f"{row['frequency']} {row['band']}<br>"
            f"{row.get('city', '')} {row.get('state', '')}<br>"
            f"ERP: {row['erp']} kW"
        )
        folium.CircleMarker(
            location=[row['lat'], lon],
            radius=6,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=220),
            tooltip=f"{row['callsign']} ({row['band']})"
        ).add_to(m)

add_markers(seahawks, '#69BE28')
add_markers(patriots, '#C60C30')

# Legend
legend_html = '''
<div style="position:fixed; bottom:30px; left:30px; z-index:1000;
            background:white; padding:10px; border:2px solid grey;
            border-radius:5px; font-size:14px;">
  <b>NFL Radio Affiliates</b><br>
  <i style="color:#69BE28;">&#11044;</i> Seahawks<br>
  <i style="color:#C60C30;">&#11044;</i> Patriots
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m

---
## Step 7 — Geographic Reach: How Far Do the Affiliates Spread?

We can measure each team's geographic footprint by computing the
distance from every affiliate to the team's home stadium using the
**Haversine formula** — the standard way to measure distance on a sphere.

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    """Great-circle distance in miles between two lat/lon points."""
    R = 3958.8  # Earth radius in miles
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Stadium coordinates
STADIUMS = {
    'Seahawks': (47.5952, -122.3316),   # Lumen Field, Seattle
    'Patriots': (42.0909,  -71.2643),   # Gillette Stadium, Foxborough
}

def add_distance(df, team_key):
    slat, slon = STADIUMS[team_key]
    df = df.copy()
    df['distance_mi'] = df.apply(
        lambda r: haversine(r['lat'], -abs(r['lon']), slat, slon)
                  if pd.notna(r['lat']) and pd.notna(r['lon']) else None,
        axis=1
    )
    return df

seahawks_d = add_distance(seahawks, 'Seahawks')
patriots_d = add_distance(patriots, 'Patriots')

sea_dist = seahawks_d['distance_mi'].dropna()
pat_dist = patriots_d['distance_mi'].dropna()

# Histogram
fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(sea_dist, bins=20, alpha=0.6, label='Seahawks', color='#69BE28', edgecolor='black')
ax.hist(pat_dist, bins=20, alpha=0.6, label='Patriots', color='#C60C30', edgecolor='black')
ax.set_xlabel('Distance from Stadium (miles)')
ax.set_ylabel('Number of Stations')
ax.set_title('How Far Do Affiliates Reach From the Home Stadium?')
ax.legend()
plt.tight_layout()
plt.show()

print(f'Seahawks — avg: {sea_dist.mean():.0f} mi, farthest: {sea_dist.max():.0f} mi')
print(f'Patriots — avg: {pat_dist.mean():.0f} mi, farthest: {pat_dist.max():.0f} mi')

# Farthest affiliates
for label, df_d in [('Seahawks', seahawks_d), ('Patriots', patriots_d)]:
    far = df_d.loc[df_d['distance_mi'].idxmax()]
    print(f'\nFarthest {label} affiliate: {far["callsign"]} {far["frequency"]} {far["band"]}'
          f' — {far["city"]}, {far["state"]} ({far["distance_mi"]:.0f} mi)')

---
## Step 8 — Summary Scorecard

Let's pull everything together into a single comparison table.

In [ ]:
def safe_mean(series):
    return f'{series.mean():.1f}' if len(series) > 0 else 'N/A'

scorecard = pd.DataFrame({
    'Metric': [
        'Total Affiliates',
        'AM Stations',
        'FM Stations',
        'AM Percentage',
        'States / Provinces',
        'Avg Distance from Stadium (mi)',
        'Farthest Affiliate (mi)',
        'Highest-Power AM (kW)',
        'Avg FM ERP (kW)',
    ],
    'Seahawks': [
        len(seahawks),
        sea_am,
        sea_fm,
        f'{sea_am / len(seahawks) * 100:.0f}%',
        seahawks['state'].nunique(),
        f'{sea_dist.mean():.0f}',
        f'{sea_dist.max():.0f}',
        f"{seahawks[seahawks['band']=='AM']['erp'].max():.0f}",
        safe_mean(seahawks[seahawks['band']=='FM']['erp'].dropna()),
    ],
    'Patriots': [
        len(patriots),
        pat_am,
        pat_fm,
        f'{pat_am / len(patriots) * 100:.0f}%',
        patriots['state'].nunique(),
        f'{pat_dist.mean():.0f}',
        f'{pat_dist.max():.0f}',
        f"{patriots[patriots['band']=='AM']['erp'].max():.0f}",
        safe_mean(patriots[patriots['band']=='FM']['erp'].dropna()),
    ],
})

scorecard_styled = scorecard.style.set_properties(**{
    'text-align': 'center',
    'font-size': '14px',
    'padding': '6px 12px',
}).set_properties(
    subset=['Metric'], **{'text-align': 'left', 'font-weight': 'bold'}
).hide(axis='index')

display(scorecard_styled)

# Quick takeaways
print()
if len(seahawks) > len(patriots):
    print(f'Seahawks lead in total affiliates: {len(seahawks)} vs {len(patriots)}')
elif len(patriots) > len(seahawks):
    print(f'Patriots lead in total affiliates: {len(patriots)} vs {len(seahawks)}')
else:
    print(f'Tied at {len(seahawks)} affiliates each!')

print(f'Seahawks: {sea_am} AM / {sea_fm} FM  |  Patriots: {pat_am} AM / {pat_fm} FM')
print(f'Seahawks cover {seahawks["state"].nunique()} states  |  Patriots cover {patriots["state"].nunique()} states')